In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import os

# Set paths to your dataset
dataset_dir = "./dataset-split"
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')
test_dir = os.path.join(dataset_dir, 'test')

# Image dimensions
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 1e-4

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest",
)

val_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Load MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Freeze base model layers
base_model.trainable = False

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

# Train the model
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=[early_stopping, reduce_lr]
)

base_model.trainable = True
for layer in base_model.layers[:60]:  
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=1e-5),  
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history_fine_tune = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    callbacks=[early_stopping, reduce_lr]
)




Found 483 images belonging to 4 classes.
Found 137 images belonging to 4 classes.
Epoch 1/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 49s 2s/step - accuracy: 0.2130 - loss: 1.8146 - val_accuracy: 0.2482 - val_loss: 1.4970 - learning_rate: 1.0000e-04
Epoch 2/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - accuracy: 0.2841 - loss: 1.6063 - val_accuracy: 0.3139 - val_loss: 1.4023 - learning_rate: 1.0000e-04
Epoch 3/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - accuracy: 0.3164 - loss: 1.5004 - val_accuracy: 0.3869 - val_loss: 1.3162 - learning_rate: 1.0000e-04
Epoch 4/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 24s 2s/step - accuracy: 0.3449 - loss: 1.4804 - val_accuracy: 0.4307 - val_loss: 1.2651 - learning_rate: 1.0000e-04
Epoch 5/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 16s 987ms/step - accuracy: 0.3777 - loss: 1.3552 - val_accuracy: 0.4526 - val_loss: 1.2301 - learning_rate: 1.0000e-04
Epoch 6/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.4415 - loss: 1.2874 - val_accuracy: 0.4672 - val_loss: 1.2051 - learning_rate: 1.

In [ ]:
import splitfolders  

# Path to the dataset containing category folders
original_dataset_dir = r"./dataset"
output_dir = r"./dataset-split"

# Split dataset into train (70%), val (20%), and test (10%)
splitfolders.ratio(
    original_dataset_dir, 
    output=output_dir, 
    seed=42, 
    ratio=(0.7, 0.2, 0.1), 
    group_prefix=None,  # Keeps images with similar prefix in the same set
    move=False  
)


In [8]:
# Evaluate the model
test_generator = val_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)



Found 74 images belonging to 4 classes.


In [9]:
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.2f}")


3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 501ms/step - accuracy: 0.4971 - loss: 1.1205
Test Accuracy: 0.49
